# Install Google Cloud Vision Client

In [2]:
%pip install --upgrade google-cloud-vision
#%pip install --upgrade google-cloud-speech
#%pip install --upgrade google-cloud-language
#%pip install --upgrade google-cloud-texttospeech

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: proto-plus>=1.15.0 in d:\users\alex\anaconda3\lib\site-packages (from google-cloud-vision) (1.19.8)



In [1]:
import os
import io
import time
import pandas as pd

# Image Locations Paths

In [3]:
# Path of the images
images_location_original="../data/raw"
images_location_facebook="../data/external"

# Path of the output
images_location_processed="../data/processed/google_API_response"


## Get list of Images Names for our own Images

In [4]:
# Downloading the csv file from your GitHub account
path=f'{images_location_original}/our_images.csv'
df = pd.read_csv(path,index_col=0)

# Extracting the list of images
images_name_list = df["Name"].tolist()

# Extracting the list of hate/noHate
hate_list = df["hate"].tolist()


# Print all images names
print(images_name_list)

['10246724_747008182064474_2873398654888745189_n.png', 'obama-meme.jpg', 'alldone.png', 'brown.png', 'patriots.png', 'pending.png', 'mikeFlynn.png', 'precipice.png', 'DUfydpRW4AA5WYF.jpg', 'DUfigmUXUAIc8G5.jpg', 'DUPUIOkXcAAujr0.jpg', 'DVKz6V5WsAE7dlR.jpg', 'hypocrite2.png', 'isaid.png', 'racistVoting.png', 'bush.png', 'pedowood.png', 'bringPain.png', 'twitterJail.jpg', 'treasonNN.jpg', 'PROSECUTION.jpg', 'faithhealer.jpg', 'PrayingMedic.jpg', 'noFlyNancy.jpg', 'maggie.png', 'internalCivilWar.jpg', 'auschwitz.jpg', '14faef5e6164585c084d65fb66d59c78.jpeg', '6fb33123bd09944b92fba03051ca17c9.jpeg', '4dfaaeb44e80b2eba0dfcc7cea091419.png', 'catwithconeears.gif', 'snowwhitesheadexplodes.gif', 'rowanatkinsononkeyboards.gif', 'arielinspired.gif', 'abesimpsoninandout.gif', 'dogstealsbra.gif', 'stuntgolfer.gif', 'wrapitup.gif', 'camerongangsta.jpg', 'aaaahhhh.gif', 'catrollsoverfallsoff.gif', 'moustachesarecool.gif', 'catattack.gif', 'bitchimfabulous.gif', 'yologangnam.gif', 'shockhorror.gif', '

## Get list of Images Names for FaceBook Images

In [5]:

# Downloading the csv file from your GitHub account
path=f'{images_location_facebook}/facebook_images.csv'
df = pd.read_csv(path,index_col=0)

# Extracting the list of images
facebook_images_name_list = df["Name"].tolist()

# Extracting the list of hate/noHate
facebook_hate_list = df["hate"].tolist()


# Print all images names
print(facebook_images_name_list)

['08291.png', '46971.png', '03745.png', '83745.png', '80243.png', '05279.png', '01796.png', '53046.png', '82301.png', '31752.png', '27635.png', '80597.png', '45368.png', '17963.png', '53968.png', '10749.png', '25149.png', '87520.png', '89071.png', '09563.png', '72048.png', '49826.png', '26453.png', '12650.png', '02568.png', '83954.png', '24316.png', '62035.png', '65342.png', '92058.png', '58672.png', '54069.png', '91586.png', '87130.png', '95640.png', '47819.png', '59140.png', '13647.png', '65832.png', '57621.png', '40982.png', '43275.png', '12973.png', '74350.png', '02478.png', '51607.png', '65403.png', '29750.png', '26547.png', '02143.png', '68192.png', '52603.png', '05349.png', '06491.png', '74013.png', '27485.png', '13026.png', '17265.png', '91405.png', '14026.png', '18945.png', '83920.png', '83497.png', '04569.png', '39607.png', '09715.png', '01925.png', '94813.png', '29437.png', '95038.png', '64891.png', '63280.png', '17682.png', '84510.png', '28690.png', '03567.png', '95830.png'

# Google Cloud Vision

In [6]:
# For the Google Cloud Vision to work, specify the location of your key

path_to_key="./memeproject-335703-018a67736bbc.json"

## Create the enviroment variable indicating the Keys file

In [7]:
# Imports Credential File:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = path_to_key
print("Service Account Key: {}".format(os.environ["GOOGLE_APPLICATION_CREDENTIALS"]))

Service Account Key: ./memeproject-335703-018a67736bbc.json


## Load the Google Vision Client

In [8]:
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()

# Auxiliary functions

### Function that gets the text from an image

In [9]:
# Get the Text From an Image
# Img: one image in the Bytes format
# Returns: Json response from Google API
def getTextFromImage(img):

  image = vision.Image(content=img)

  response = client.text_detection(image=image)  # returns TextAnnotation JSON format
  # texts = response.text_annotations # Convert the JSON into a List JSON elements

  return(response)

### Function that gets the objects from an image

In [10]:
# Get the Objects from an Image
# Img: one image in the Bytes format
# Returns: Json response from Google API
def getObjectsFromImage(img):

  image = vision.Image(content=img)

  response = client.object_localization(image=image)  # returns TextAnnotation JSON format
  # texts = response.text_annotations # Convert the JSON into a List JSON elements

  return(response)

### Process all images

Function that read the image

In [11]:
# Open the Image specify by the "path_images" and the "name" of the image. It returns it as a Bytes type
# path_images: path of the folder that contains the image
# name: name of the image that we wish to open
# Returns: Content of the Image in Bytes Format
def open_img(path_images,name):
  # Create URL of Image
  url=f'{path_images}/{name}'
  #print("Image Name: ",url)

  # Open Image
  with io.open(url, 'rb') as image_file:
    content = image_file.read()
        
  return(content)

Function that save the response

In [12]:
# Method that save the response into the corresponding file.
# name_file: name of csv file where the content is saved
# content_names: list of all the names of the images 
# content_hate: list of all the categoriacal casiflication for all the images (0: No hate | 1: Hate)
# content_text: list of all the json responses from the getText API function
# content_objects: list of all the json responses from the getObjects API function
def save_response(name_file,content_names,content_hate,content_text,content_objects):
  #Create the route for the save file
  path_to_file=f'{images_location_processed}/{name_file}.csv' 

  # Create the df
  data = {'Name':content_names,
          'hate':content_hate,
          'text':content_text,
          'object':content_objects}
  df = pd.DataFrame(data)
  
  # Save the dataframe as a csv file
  df.to_csv(path_to_file, index=False)

# Main Loop

In [13]:
# Reiterate over all images an get the text and objects
# origin specify which dataset to use --> If 'our' it will processed our memes. If 'facebook' it will processed facebook memes.
def mainGetTextAndObjectsFromImages(origin):
  # Create Timer to measure time spent
  start = time.time()
  print("Start time: ",start)

  
  # Check if we are using our own dataset or facebook dataset
  dataset=[]
  dataset_hate=[]
  path_open=""
  if(origin=="our"):
    dataset=images_name_list
    dataset_hate=hate_list
    path_open=images_location_original
  elif(origin=="facebook"):
    dataset=facebook_images_name_list
    dataset_hate= facebook_hate_list
    path_open=images_location_facebook
  else:
    print("ERROR: Choose 'our' or 'facebook' dataset")
    return (1)
  
  print("Path: ",path_open)
  #print("Images: ",dataset)
  
  # list to save the responses
  list_text=[]
  list_object=[]
  
  count=1
  # Loop all the Images
  for image_name in dataset:
    # Print iteration and avg time spent
    print_count=50 # Counter, Prints itself each print_count
    if(count % print_count==0):
      print("Iteration: " + str(count) + " | Avg Time/img: " + str((time.time()-start)/count))
    count+=1


    # Open Image
    content=open_img(path_open,image_name)

    # Proccess Image
    # Get Text
    result_text = getTextFromImage(content)
    # Get Objects
    result_objects = getObjectsFromImage(content)

    # Save responses
    list_text.append(result_text)
    list_object.append(result_objects)

    #break
  
  # Save to file
  #print("list text: ",list_text)
  #print("list object: ",list_object)
  save_response(f'{origin}_images_API_response',dataset,dataset_hate,list_text,list_object)
  
  end = time.time()
  print("End time: ",end)
  print("Time elapsed (sec): ",end - start)
  print("Avg Time/img: " + str((end-start)/count))

## Run Everything

In [66]:
# Run the code for "our" own memes
mainGetTextAndObjectsFromImages("our")

Start time:  1640699806.246478
Path:  ../data/raw
Iteration: 20 | Avg Time/img: 3.1469215631484984
Iteration: 40 | Avg Time/img: 2.7817333161830904
Iteration: 60 | Avg Time/img: 2.9542929848035175
Iteration: 80 | Avg Time/img: 2.851920560002327
Iteration: 100 | Avg Time/img: 2.850111794471741
Iteration: 120 | Avg Time/img: 2.831601725021998
Iteration: 140 | Avg Time/img: 2.726032326902662
Iteration: 160 | Avg Time/img: 2.5945176064968107
Iteration: 180 | Avg Time/img: 2.4223732577429877
Iteration: 200 | Avg Time/img: 2.404889520406723
Iteration: 220 | Avg Time/img: 2.280617496100339
Iteration: 240 | Avg Time/img: 2.167714564005534
Iteration: 260 | Avg Time/img: 2.067071215923016
Iteration: 280 | Avg Time/img: 1.9852360308170318
Iteration: 300 | Avg Time/img: 1.9272727926572164
Iteration: 320 | Avg Time/img: 1.8645831286907195
Iteration: 340 | Avg Time/img: 1.8311321097261766
Iteration: 360 | Avg Time/img: 1.780944996409946
Iteration: 380 | Avg Time/img: 1.741428926743959
Iteration: 400

In [14]:
# Run the code for "facebook" memes
mainGetTextAndObjectsFromImages("facebook")

Start time:  1640719420.2395287
Path:  ../data/external
Iteration: 50 | Avg Time/img: 1.8825875186920167
Iteration: 100 | Avg Time/img: 1.8559522652626037
Iteration: 150 | Avg Time/img: 1.8192328484853109
Iteration: 200 | Avg Time/img: 1.9097770726680756
Iteration: 250 | Avg Time/img: 1.9982937517166137
Iteration: 300 | Avg Time/img: 1.9321130243937175
Iteration: 350 | Avg Time/img: 1.8991955300739833
Iteration: 400 | Avg Time/img: 1.8725056844949721
Iteration: 450 | Avg Time/img: 1.8478147432539198
Iteration: 500 | Avg Time/img: 1.8301431121826173
Iteration: 550 | Avg Time/img: 1.803751553622159
Iteration: 600 | Avg Time/img: 1.7946384390195211
Iteration: 650 | Avg Time/img: 1.773300670843858
Iteration: 700 | Avg Time/img: 1.7425605038234167
Iteration: 750 | Avg Time/img: 1.7303692344029744
Iteration: 800 | Avg Time/img: 1.719932306110859
Iteration: 850 | Avg Time/img: 1.7106123511931475
Iteration: 900 | Avg Time/img: 1.6983940516577827
Iteration: 950 | Avg Time/img: 1.686507667742277

# Since the resulting file is to big to upload it to GitHub. I will split it into diferent files

Due to the size of the file, its imposible to upload it to the github. For that the csv file will be split into smallers ones

In [18]:
# Open the save dataframe
df = pd.read_csv (f'{images_location_processed}/facebook_images_API_response.csv')

In [19]:
df

,Name,hate,text,object
0,08291.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
1,46971.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/0jy..."
2,03745.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
3,83745.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/j/5qg..."
4,80243.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
...,...,...,...,...
12135,10423.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
12136,98203.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
12137,36947.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."
12138,16492.png,1,"text_annotations {\n locale: ""en""\n descript...","localized_object_annotations {\n mid: ""/m/01g..."


In [20]:
import numpy as np
# split the dataframe into 10 smaller diferent ones
df_list=np.array_split(df, 10)

In [25]:
for i in df_list:
    print(i)

           Name  hate                                               text  \
0     08291.png     1  text_annotations {\n  locale: "en"\n  descript...   
1     46971.png     1  text_annotations {\n  locale: "en"\n  descript...   
2     03745.png     1  text_annotations {\n  locale: "en"\n  descript...   
3     83745.png     1  text_annotations {\n  locale: "en"\n  descript...   
4     80243.png     1  text_annotations {\n  locale: "en"\n  descript...   
...         ...   ...                                                ...   
1209  13074.png     0  text_annotations {\n  locale: "en"\n  descript...   
1210  80653.png     0  text_annotations {\n  locale: "en"\n  descript...   
1211  27408.png     1  text_annotations {\n  locale: "en"\n  descript...   
1212  13726.png     0  text_annotations {\n  locale: "en"\n  descript...   
1213  61792.png     0  text_annotations {\n  locale: "en"\n  descript...   

                                                 object  
0     localized_object_annota

In [26]:
#Save into 10 diferent datasets
path_to_file=f'{images_location_processed}/facebook_images_API_response' 

count=1
for i in df_list:
    # Save the dataframe as a csv file
    df.to_csv(f'{path_to_file}_{count}.csv', index=False)
    # Update Counter
    count+=1

# Google Cloud Vision Functions:


Text detection: **text_detection()**
https://cloud.google.com/vision/docs/ocr

For documents and handwriten text:  **document_text_detection()**
https://cloud.google.com/vision/docs/handwriting

Detect Faces: **face_detection()** 
https://cloud.google.com/vision/docs/detecting-faces

Get colors: **image_properties()**
https://cloud.google.com/vision/docs/detecting-properties

Detect Tags of an Image: **label_detection()**
https://cloud.google.com/vision/docs/labels


Detect Landmarks: **landmark_detection()**
https://cloud.google.com/vision/docs/detecting-landmarks

Detect Logos: **logo_detection()**
https://cloud.google.com/vision/docs/detecting-logos

Detect Object and position: **object_localization()**
https://cloud.google.com/vision/docs/object-localizer

Detect explicit Content: **safe_search_detection()**
https://cloud.google.com/vision/docs/detecting-safe-search

Detect Web Pages: **web_detection()**
https://cloud.google.com/vision/docs/detecting-web